In [1]:
#@title First we'll need to import the LangChain x OpenAI integration package.
!pip install -qU langchain
!pip install -qU cohere langchain-openai

In [2]:
from langchain.chains import create_extraction_chain
from langchain_openai import ChatOpenAI, OpenAI

from langchain.prompts import PromptTemplate

from langchain_core.pydantic_v1 import(
    BaseModel,
    Field,
    root_validator,
    Json
)

from langchain.chains import create_extraction_chain_pydantic
from langchain.output_parsers import PydanticOutputParser

from typing import Optional, List, Sequence

import json
import pprint
import math

In [3]:
#@title Accessing the API requires an API key
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

model = OpenAI(temperature=0)

### Pydantic

Pydantic is a data validation and settings management library for Python.

It allows you to create data classes with attributes that are automatically validated when you instantiate an object.

Lets define a class with attributes annotated with types.

In [4]:
#@title Pydantic data class

class Temperature(BaseModel):
    # description: The fluid temperature.
    value: float = Field(default= 298.15)
    units: str = Field(default= 'Kelvin')

class Elevation_above_sea_level(BaseModel):
    # Description: The elevation above sea level is where the control valve is located.
    value: float = Field(default= 0.0)
    unit: str = Field(default= 'meter')

    @root_validator
    def ensure_value_is_non_negative(cls, values):
        value = values.get('value')
        values['value'] = math.fabs(value)  # Calculate absolute value
        return values

class SizeUpstreamPipe(BaseModel):
    # Description: The diameter of the pipe upstream of the control valve.
    value: Optional[float] = None
    unit: str

    @root_validator
    def ensure_value_is_non_negative(cls, values):
        value = values.get('value')
        values['value'] = math.fabs(value)  # Calculate absolute value
        return values

class SizeValve(BaseModel):
    # Description: The diameter of the control valve.
    value: Optional[float] = None
    unit: str

    @root_validator
    def ensure_value_is_non_negative(cls, values):
        value = values.get('value')
        values['value'] = math.fabs(value)  # Calculate absolute value
        return values

class SizeDownstreamPipe(BaseModel):
    # Description: The diameter of the pipe downstream of the control valve.
    value: Optional[float] = None
    unit: str

    @root_validator
    def ensure_value_is_non_negative(cls, values):
        value = values.get('value')
        values['value'] = math.fabs(value)  # Calculate absolute value
        return values

class PressureInlet(BaseModel):
    # Description: The inlet pressure, or upstream pressure, of the control valve.
    value: Optional[float] = None
    unit: str
    measurement: str

    @root_validator
    def ensure_value_is_non_negative(cls, values):
        value = values.get('value')
        values['value'] = math.fabs(value)  # Calculate absolute value
        return values

class PressureOutlet(BaseModel):
    # Description: The outlet pressure, or downstream pressure, of the control valve.
    value: Optional[float] = None
    unit: str
    measurement: str

    @root_validator
    def ensure_value_is_non_negative(cls, values):
        value = values.get('value')
        values['value'] = math.fabs(value)  # Calculate absolute value
        return values

class Flow_Rate(BaseModel):
    # Description: Required flow under upstream and downstream pressure conditions.
    value: Optional[float] = None
    unit: str

    @root_validator
    def ensure_value_is_non_negative(cls, values):
        value = values.get('value')
        values['value'] = math.fabs(value)  # Calculate absolute value
        return values

class Pressure(BaseModel):
    inlet: PressureInlet
    outlet: PressureOutlet

class Size(BaseModel):
    upstream_pipe: SizeUpstreamPipe
    valve: SizeValve
    downstream_pipe: SizeDownstreamPipe

class Sizing_Data(BaseModel):
    fluid: Optional[str] = None
    temperature: Temperature
    height_above_sea_level: Elevation_above_sea_level
    size: List[Size] = []
    pressure: List[Pressure] = []
    flow_rate: List[Flow_Rate] = []

In [5]:
#@title Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Sizing_Data)

In [6]:
#@title prompt_template

prompt_template = """
  You are a highly skilled engineering consultant who specializes in sizing control
  valves. You should extract structured information from the user's query that
  matches the format described below. Make sure the extracted information matches
  the information type you are looking for. Use only the attributes listed in the
  schema without adding any additional attributes.
  \n{format_instructions}\n{query}\n"""

In [7]:
#@title Prompt
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [8]:
#@title User Query

query = """
Fluid: Water.
Altitude: 1500 m.
Inlet Temperature: 295.15K.
Inlet Absolute Pressure: 680 kPa.
Outlet Absolute Pressure: 220 kPa.
Flow Rate (Q): 360 m3/h.
Pipe upstream Size: 150mm.
valve Size: 100mm.
Pipe downstream Size: 100mm.
"""


In [9]:
#@title Run
_input = prompt.format_prompt(query=query)
output = model(_input.to_string())
output = parser.parse(output)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [10]:
#@title Print Pydantic class as JSON
print(output.json(indent=2))

{
  "fluid": "Water",
  "temperature": {
    "value": 295.15,
    "units": "Kelvin"
  },
  "height_above_sea_level": {
    "value": 1500.0,
    "unit": "meter"
  },
  "size": [
    {
      "upstream_pipe": {
        "value": 150.0,
        "unit": "mm"
      },
      "valve": {
        "value": 100.0,
        "unit": "mm"
      },
      "downstream_pipe": {
        "value": 100.0,
        "unit": "mm"
      }
    }
  ],
  "pressure": [
    {
      "inlet": {
        "value": 680.0,
        "unit": "kPa",
        "measurement": "absolute"
      },
      "outlet": {
        "value": 220.0,
        "unit": "kPa",
        "measurement": "absolute"
      }
    }
  ],
  "flow_rate": [
    {
      "value": 360.0,
      "unit": "m3/h"
    }
  ]
}
